# 機械学習帳 確認問題(8.4, 8.5)

[機械学習帳のニューラルネットワーク(2)の確認問題(8.4, 8.5)](https://chokkan.github.io/mlnote/classification/04nntrain.html#id13)を解きます。

## 講義内容の復習

![chapter8-1.svg](./images/chapter8/chapter8-1.svg)

![chapter8-2.svg](./images/chapter8/chapter8-2.svg)

## 8.4 自動微分

2次元の入力$\pmb{x} \in \mathbb{R}^{2}$を受け取り、2次元の中間層$\pmb{h} \in \mathbb{R}^{2}$を介して、$\hat{y} \in \mathbb{R}$を出力する2層のニューラルネットワークを次式で定義する。
\begin{align*}
\hat{y} = \sigma(\pmb{h}^{\top}\pmb{q}+c), \; \pmb{h}=\mathrm{ReLU}(\pmb{W}\pmb{x} + \pmb{b})
\end{align*}
ただし、$\sigma(.)$はシグモイド関数であり、$\mathrm{ReLU}(a) = \max(a, 0)$である。また、$\pmb{W} \in \mathbb{R}^{2 \times 2}, \pmb{q} \in \mathbb{R}^{2}, b \in \mathbb{R}^2, c \in \mathbb{R}$はモデルのパラメータであり、以下に示す値に設定されているとする。
\begin{align*}
\pmb{W} = \begin{pmatrix}1 & 1 \\ -1 & -1\end{pmatrix}, \; \pmb{q} = \begin{pmatrix}1 \\ 1\end{pmatrix}, \; \pmb{b} = \begin{pmatrix}-0.5 \\ 1.5\end{pmatrix}, \; c = -1.5
\end{align*}

### 8.4.1 出力の計算

このモデルに入力$\pmb{x}=\begin{pmatrix}0 \\ 0\end{pmatrix}$が与えられたとき、中間層の値$\pmb{h}$と出力値$\hat{y}$を求めよ。

### 8.4.2 勾配の計算

ある学習事例$(\pmb{x}, y) = \left(\begin{pmatrix}1 \\ 1 \end{pmatrix}, 0\right)$に対して、このニューラルネットワークをクロスエントロピー損失関数の値、および損失関数の$\pmb{W}$, $\pmb{q}$, $\pmb{b}$, $c$に関する勾配の値を求めよ。

## 8.5 NNの学習

### 8.5.1 モデルの学習

MNISTの訓練データを用いて単層のニューラルネットワークを学習せよ。このとき、各エポック毎に以下の評価値を計算し、グラフとして表示せよ。

+ 訓練データ上の損失関数の値
+ 訓練データ上の正解率
+ 評価データ上の損失関数の値
+ 評価データ上の正解率

#### データの準備

MNISTのデータを準備します(6章で使用したコードを再掲)

In [1]:
import gzip
import sys
import struct
import urllib.request
from pathlib import Path
import numpy as np

def read_image(fi):
    magic, n, rows, columns = struct.unpack(">IIII", fi.read(16))
    assert magic == 0x00000803
    assert rows == 28
    assert columns == 28
    rawbuffer = fi.read()
    assert len(rawbuffer) == n * rows * columns
    rawdata = np.frombuffer(rawbuffer, dtype='>u1', count=n*rows*columns)
    return rawdata.reshape(n, rows, columns).astype(np.float32) / 255.0

def read_label(fi):
    magic, n = struct.unpack(">II", fi.read(8))
    assert magic == 0x00000801
    rawbuffer = fi.read()
    assert len(rawbuffer) == n
    return np.frombuffer(rawbuffer, dtype='>u1', count=n)

def openurl_gzip(url):
    request = urllib.request.Request(
        url,
        headers={
            "Accept-Encoding": "gzip",
            "User-Agent": "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11", 
        })
    response = urllib.request.urlopen(request)
    return gzip.GzipFile(fileobj=response, mode='rb')

def save_mnist():
    if Path("data/mnist.npz").exists():
        return
    np.savez_compressed(
        "data/mnist",
        train_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")),
        train_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")),
        test_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")),
        test_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"))
    )

save_mnist()
data = np.load("data/mnist.npz")
print("Training data (X):", data["train_x"].shape, data["train_x"].dtype)
print("Training data (Y):", data["train_y"].shape, data["train_y"].dtype)
print("Test data (X):", data["test_x"].shape, data["test_x"].dtype)
print("Test data (Y):", data["test_y"].shape, data["test_y"].dtype)

Training data (X): (60000, 28, 28) float32
Training data (Y): (60000,) uint8
Test data (X): (10000, 28, 28) float32
Test data (Y): (10000,) uint8


#### モデルの学習

### 8.5.2 適合率、再現率、F1スコア

(1)で学習したモデルに対して、評価データ上で以下の評価値を計算せよ。

+ 各カテゴリ（数字）に対する適合率、再現率、F1スコア
+ 各カテゴリの適合率、再現率、F1スコアのマクロ平均を取ったもの

### 8.5.3 混同行列

(1)で学習したモデルに対して、評価データ上で混同行列を計算せよ。

### 8.5.4 紛らわしい事例

(1)で学習したモデルにとって認識が簡単な評価事例トップ3と、難しい評価事例トップ3を求めたい。ある事例$x_i$に対して、正解のカテゴリを$y_i$、モデルが予測したカテゴリを$\hat{y}_i$とする。モデルが認識に成功した（つまり$\hat{y}_i = y_i$となる）事例に対して、認識の簡単さを条件付き確率$p(\hat{y} | x)$で定義する。一方で、モデルが認識に失敗した（つまり$\hat{y}_i \neq y_i$となる）事例に対しても、認識の難しさを条件付き確率$p(\hat{y} | x)$で定義する。

このようにして見つけた6件の評価事例のそれぞれに対して、$y_i$, $p(y_i | x_i)$, $\hat{y}_i$, $p(\hat{y}_i | x_i)$とその事例の画像を表示せよ。

### 8.5.5 多層ニューラルネットワーク

多層ニューラルネットワークを適当に構成し、(1)の実験を再度実行せよ。どのようなネットワーク構成にするかは任意である。